In [1]:
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, OneHotEncoder
from sklearn.feature_selection import SelectKBest,RFE, chi2
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from fancyimpute import IterativeImputer
import xgboost as xgb
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
train  = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [3]:
train.shape

(1460, 81)

In [4]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [5]:
def fill_ii(df):
    df_filled_ii = pd.DataFrame(IterativeImputer().fit_transform(df.values))
    df_filled_ii.columns = df.columns
    df_filled_ii.index = df.index

    return df_filled_ii

In [6]:
train["SalePrice"] = np.log1p(train["SalePrice"])
y = train['SalePrice']

train = train.drop(['Id', 'SalePrice','Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)


# One-Hot encode all data
train = pd.get_dummies(train, prefix_sep='_')

# Impute all data, using IterativeImputer
train = fill_ii(train)


In [8]:
results = []
NUM_TRIALS = 50
for trial in range(NUM_TRIALS):
    print('Running {0} / {1}'.format(trial,NUM_TRIALS))
    test_size = 0.33
    seed = 7
    X_train, X_test, Y_train, Y_test = train_test_split(train, y, test_size=test_size, random_state=seed)
    model = LinearRegression() 
    model.fit(X_train, Y_train)
    result = model.score(X_test, Y_test) 
    RMSLE = np.sqrt(mean_squared_error(Y_test,model.predict(X_test)))
    results.append(RMSLE)
    print("Accuracy: %.3f%%" % (result*100.0))
print(results)

Running 0 / 50
Accuracy: 84.859%
Running 1 / 50
Accuracy: 84.859%
Running 2 / 50
Accuracy: 84.859%
Running 3 / 50
Accuracy: 84.859%
Running 4 / 50
Accuracy: 84.859%
Running 5 / 50
Accuracy: 84.859%
Running 6 / 50
Accuracy: 84.859%
Running 7 / 50
Accuracy: 84.859%
Running 8 / 50
Accuracy: 84.859%
Running 9 / 50
Accuracy: 84.859%
Running 10 / 50
Accuracy: 84.859%
Running 11 / 50
Accuracy: 84.859%
Running 12 / 50
Accuracy: 84.859%
Running 13 / 50
Accuracy: 84.859%
Running 14 / 50
Accuracy: 84.859%
Running 15 / 50
Accuracy: 84.859%
Running 16 / 50
Accuracy: 84.859%
Running 17 / 50
Accuracy: 84.859%
Running 18 / 50
Accuracy: 84.859%
Running 19 / 50
Accuracy: 84.859%
Running 20 / 50
Accuracy: 84.859%
Running 21 / 50
Accuracy: 84.859%
Running 22 / 50
Accuracy: 84.859%
Running 23 / 50
Accuracy: 84.859%
Running 24 / 50
Accuracy: 84.859%
Running 25 / 50
Accuracy: 84.859%
Running 26 / 50
Accuracy: 84.859%
Running 27 / 50
Accuracy: 84.859%
Running 28 / 50
Accuracy: 84.859%
Running 29 / 50
Accuracy